# Forcing CPU training

In [1]:
import tensorflow as tf

# listing all physical devices
physical_devices = tf.config.list_physical_devices('GPU')

# disabling all GPU
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, False)
    tf.config.set_visible_devices([], 'GPU')

2025-01-25 11:13:03.387134: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 11:13:03.395753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-25 11:13:03.408429: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-25 11:13:03.408455: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 11:13:03.416711: I tensorflow/core/platform/cpu_feature_gua

# Loading data from preprocessed pickle file

In [2]:
import pickle

# open the pickle file in read-binary mode
with open('..//model-data/preprocessing/train-val-sets/train_val_split.pkl', 'rb') as f:
    data = pickle.load(f)

# storing data
images_train = data['images_train']
images_val = data['images_val']
labels_train = data['labels_train']
labels_val = data['labels_val']

# Creating model

In [3]:
import tensorflow as tf
import keras
from keras import layers, Sequential

model = keras.Sequential([
    layers.Input(shape=(256, 256, 3)),
    layers.Flatten(),
    layers.Dense(39, activation='softmax')
])

# Compling the model

In [4]:
# compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fitting the model

In [5]:
model.fit(images_train, labels_train, epochs=5)

Epoch 1/5
   1/1321 ━━━━━━━━━━━━━━━━━━━━ 9:50 447ms/step - accuracy: 0.0625 - loss: 348.0638

I0000 00:00:1737803595.485095    2566 service.cc:145] XLA service 0x78b0ec0085b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737803595.485128    2566 service.cc:153]   StreamExecutor device (0): Host, Default Version
2025-01-25 11:13:15.493806: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737803595.608647    2566 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1321/1321 ━━━━━━━━━━━━━━━━━━━━ 23s 17ms/step - accuracy: 0.2560 - loss: 19733.1328
Epoch 2/5
1321/1321 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.4203 - loss: 7296.1973
Epoch 3/5
1321/1321 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.4684 - loss: 7517.5020
Epoch 4/5
1321/1321 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.4960 - loss: 7294.3652
Epoch 5/5
1321/1321 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.5196 - loss: 7111.2480


# Validating the model

In [6]:
import numpy as np
from sklearn.metrics import accuracy_score

# converting one hot encoded labels to class indices
real_labels = np.argmax(labels_val, axis=1)

# predict on the validation set
predictions = model.predict(images_val)

# converting predictions from one hot encoded probabilities to class indices
predicted_labels = np.argmax(predictions, axis=1)

# comparing predictions with the real labels
print("Validation Accuracy:", accuracy_score(real_labels, predicted_labels))

331/331 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Validation Accuracy: 0.5040242401287757


In [12]:
print(predictions[10])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [13]:
print(labels_val[10])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
